In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ipython_sql

In [ ]:
%load_ext sql

In [ ]:
import sqlite3

In [ ]:
conn=sqlite3.connect('database.db')

In [ ]:
%sql sqlite:///database.db

In [ ]:
import pandas as pd

In [ ]:
imdb_data= pd.read_csv('/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv')

In [ ]:
imdb_data.to_sql('imdb_data',conn,if_exists='replace',index=False)

In [ ]:
%%sql
UPDATE imdb_data
SET Gross = 0
WHERE Gross IS NULL;

In [ ]:
%%sql
UPDATE imdb_data
SET Gross=CAST(REPLACE(Gross, ',', '') AS INTEGER);

In [ ]:
%%sql
SELECT
    Director,
    Gross,
    RANK() OVER (ORDER BY Gross DESC) AS Gross_rating
FROM
    imdb_data
ORDER BY
    Gross_rating ASC
LIMIT 20;

In [ ]:
%%sql
SELECT 
    Released_year,
    Genre,
    COUNT(*) AS genre_count,
    SUM(Gross) AS total_earning
FROM
    imdb_data
GROUP BY
    Genre
ORDER BY
    genre_count DESC;

In [ ]:
%%sql
SELECT 
     Series_Title,
     Gross,
     imdb_rating
FROM
     imdb_data;

In [ ]:
%%sql
WITH year_over_year_ratings AS (
  SELECT 
        Genre,
        Released_Year,
        COUNT(*) AS Genre_count,
        AVG(IMDB_rating) AS average_rating,
        LAG(AVG(IMDB_rating)) OVER (PARTITION BY Genre ORDER BY Released_Year) AS prevyear_rating
  FROM 
      imdb_data
  GROUP BY 
    Genre,
    Released_Year
)
SELECT
     Genre,
     Released_Year,
     Genre_count,
     average_rating,
     ((average_rating-prevyear_rating)/prevyear_rating)*100 AS percentage_change_of_rating
FROM 
     year_over_year_ratings
WHERE 
     prevyear_rating IS NOT NULL;

In [ ]:
%%sql
UPDATE imdb_data
SET Runtime = CAST(REPLACE(Runtime, 'min','') AS REAL);

In [ ]:
%%sql
SELECT
    CASE 
       WHEN Runtime > 45 THEN 'Very Short'
       WHEN Runtime >= 80 AND Runtime <= 100 THEN 'Short'
       WHEN Runtime >= 101 AND Runtime <= 120 THEN 'Medium'
       WHEN Runtime >= 121 AND Runtime <= 140 THEN 'Long'
       ELSE 'Very Long'
    END AS movie_length,
    COUNT(*) AS movie_count,
    AVG(Gross) AS avg_gross
FROM
    imdb_data
GROUP BY
    movie_length;

In [ ]:
%%sql
WITH ActorData AS (
  SELECT Star1, Star2, Star3, Star4
  FROM imdb_data
)
SELECT
    Actor,
    SUM(Gross) AS total_earnings,
    AVG(Gross) AS avg_earnings,
    COUNT(*) AS movie_count,
    RANK() OVER (ORDER BY SUM(Gross) DESC) AS Actor_Rank
FROM (
    SELECT Star1 AS Actor FROM ActorData
    UNION ALL
    SELECT Star2 FROM ActorData
    UNION ALL
    SELECT Star3 FROM ActorData
    UNION ALL
    SELECT Star4 FROM ActorData
LIMIT 20
) AS ActorList
JOIN
    imdb_data ON ActorList.Actor = imdb_data.Star1 OR
                     ActorList.Actor = imdb_data.Star2 OR
                     ActorList.Actor = imdb_data.Star3 OR
                     ActorList.Actor = imdb_data.Star4
GROUP BY
    Actor

In [ ]:
%%sql
SELECT
     Released_year,
     SUM(Gross) AS total_earning,
     COUNT(*) AS movies_released
FROM
    imdb_data
GROUP BY
    Released_Year
ORDER BY
    total_earning DESC;